In [71]:
import pandas as pd
from sklearn.pipeline import Pipeline
from feature_engine.outliers import Winsorizer, OutlierTrimmer
from category_encoders import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler,PolynomialFeatures
from sklearn.model_selection import train_test_split,GridSearchCV
import lightgbm as lgb
from sklearn.impute import SimpleImputer


In [39]:
app_train = pd.read_csv("../CreditRisk/home-credit-default-risk/application_train.csv")
app_test = pd.read_csv("../CreditRisk/home-credit-default-risk/application_test.csv")

In [40]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
 app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])

In [42]:
label_encoder_vars = [col for col in app_train.select_dtypes("object").columns if len(app_train[col].unique()) <= 2]

In [43]:
dummies_vars = [col for col in app_train.select_dtypes("object").columns if len(app_train[col].unique()) > 2]

In [44]:
numerical_vars = [col for col in app_train.select_dtypes("number").columns if col not in ["SK_ID_CURR", "TARGET"]]

In [45]:
X_treino = app_train.drop(["SK_ID_CURR", "TARGET"], axis=1)
y_treino = app_train["TARGET"]

In [157]:
import numpy as np
# dummies
dummies_pipe = Pipeline(steps=[    
  ('one_hot_encoder', OneHotEncoder())
])

# ordinal encoder
ordinal_encoder_pipe = Pipeline(steps=[
  ("label_encoder", OrdinalEncoder())
])

# standard scaler
numerical_pipe = Pipeline(steps=[
  ("standard_scaler", StandardScaler())
])

#simple imputer
simple_imputer = Pipeline(steps=[
     ("imputer_median", SimpleImputer(missing_values=np.nan,strategy = 'median'))
])

# Polynomial Features
polynomial_pipe = Pipeline(steps=[
 
  ("polynomial_pipe", PolynomialFeatures(degree = 3))
])

In [161]:
poly_colums = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']
# column transformer
column_transformer = ColumnTransformer(transformers=[    
    ("cat_label_encoder", ordinal_encoder_pipe, label_encoder_vars),
    ("cat_dummies", dummies_pipe, dummies_vars),
    ("numerical", numerical_pipe, numerical_vars),
    ("simple_imputer",simple_imputer,poly_colums),   
    ("polynomial",polynomial_pipe,poly_colums),
])
column_transformer = column_transformer.fit(X_treino)

/home/raphael/anaconda3/envs/datascience/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [145]:
X_treino_transf = column_transformer.transform(X_treino)

In [151]:
()

AttributeError: Transformer cat_label_encoder (type Pipeline) does not provide get_feature_names.

In [14]:

hyper={'n_estimators':range(1,6),}
gd=GridSearchCV(estimator=lgb.LGBMClassifier(),param_grid=hyper,verbose=10,)
gd.fit(X_treino_tf,y_treino)
print(gd.best_score_)
print(gd.best_estimator_)



# lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
#                                    class_weight = 'balanced', learning_rate = 0.05, 
#                                    reg_alpha = 0.1, reg_lambda = 0.1, 
#                                    subsample = 0.8, n_jobs = -1, random_state = 50)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5; 1/5] START n_estimators=1..............................................
[CV 1/5; 1/5] END ............................n_estimators=1; total time=   1.4s
[CV 2/5; 1/5] START n_estimators=1..............................................
[CV 2/5; 1/5] END ............................n_estimators=1; total time=   1.2s
[CV 3/5; 1/5] START n_estimators=1..............................................
[CV 3/5; 1/5] END ............................n_estimators=1; total time=   1.2s
[CV 4/5; 1/5] START n_estimators=1..............................................
[CV 4/5; 1/5] END ............................n_estimators=1; total time=   1.2s
[CV 5/5; 1/5] START n_estimators=1..............................................
[CV 5/5; 1/5] END ............................n_estimators=1; total time=   1.2s
[CV 1/5; 2/5] START n_estimators=2..............................................
[CV 1/5; 2/5] END ............................n_e

In [28]:
X_text = column_transformer.transform(app_test.drop(["SK_ID_CURR"],axis=1))

In [33]:
app_test["TARGET"] = gd.best_estimator_.predict_proba(X_text)[:,1]

In [34]:
submission = app_test[["SK_ID_CURR","TARGET"]]

In [35]:
submission.head()

,SK_ID_CURR,TARGET
0,100001,0.080334
1,100005,0.080178
2,100013,0.075939
3,100028,0.075939
4,100038,0.083936


In [36]:
submission.to_csv("submission_21_03.csv", index=False)